In [1]:
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [2]:
data_train_path = 'Deepfake/Train'
data_test_path = 'Deepfake/Test'
data_val_path = 'Deepfake/Validation'

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


In [4]:

train_flow = train_datagen.flow_from_directory(data_train_path, target_size=(64, 64), batch_size=32, class_mode='categorical')
validation_flow = validation_datagen.flow_from_directory(data_val_path, target_size=(64, 64), batch_size=32, class_mode='categorical')
test_flow = test_datagen.flow_from_directory(data_test_path, target_size=(64, 64), batch_size=32, class_mode='categorical')

Found 20038 images belonging to 2 classes.
Found 4012 images belonging to 2 classes.
Found 4012 images belonging to 2 classes.


In [5]:
from keras.layers import Dropout

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25)) # Dropout layer after pooling
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25)) 
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25)) 
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5)) 
model.add(Dense(2, activation='softmax'))


C:\Users\DELL\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:

model.fit(train_flow, epochs=10, validation_data=validation_flow)

Epoch 1/10


C:\Users\DELL\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


627/627 ━━━━━━━━━━━━━━━━━━━━ 85s 131ms/step - accuracy: 0.6487 - loss: 0.6127 - val_accuracy: 0.7684 - val_loss: 0.4606
Epoch 2/10
627/627 ━━━━━━━━━━━━━━━━━━━━ 65s 103ms/step - accuracy: 0.8102 - loss: 0.4175 - val_accuracy: 0.7846 - val_loss: 0.4177
Epoch 3/10
627/627 ━━━━━━━━━━━━━━━━━━━━ 65s 103ms/step - accuracy: 0.8401 - loss: 0.3657 - val_accuracy: 0.8784 - val_loss: 0.3040
Epoch 4/10
627/627 ━━━━━━━━━━━━━━━━━━━━ 65s 103ms/step - accuracy: 0.8657 - loss: 0.3197 - val_accuracy: 0.8208 - val_loss: 0.3618
Epoch 5/10
627/627 ━━━━━━━━━━━━━━━━━━━━ 66s 105ms/step - accuracy: 0.8778 - loss: 0.2916 - val_accuracy: 0.8814 - val_loss: 0.2611
Epoch 6/10
627/627 ━━━━━━━━━━━━━━━━━━━━ 66s 104ms/step - accuracy: 0.8855 - loss: 0.2816 - val_accuracy: 0.8819 - val_loss: 0.2654
Epoch 7/10
627/627 ━━━━━━━━━━━━━━━━━━━━ 66s 105ms/step - accuracy: 0.8912 - loss: 0.2555 - val_accuracy: 0.9020 - val_loss: 0.2164
Epoch 8/10
627/627 ━━━━━━━━━━━━━━━━━━━━ 67s 106ms/step - accuracy: 0.9026 - loss: 0.2399 - val

In [8]:
loss, accuracy = model.evaluate(test_flow)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

126/126 ━━━━━━━━━━━━━━━━━━━━ 11s 88ms/step - accuracy: 0.8694 - loss: 0.3119
Test Accuracy: 86.24%


In [9]:
loss, accuracy = model.evaluate(train_flow)
print(f'Train Accuracy: {accuracy * 100:.2f}%')

627/627 ━━━━━━━━━━━━━━━━━━━━ 32s 50ms/step - accuracy: 0.9390 - loss: 0.1629
Train Accuracy: 93.83%


In [36]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('fake_img.jpeg', target_size=(64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
train_flow.class_indices

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step


{'Fake': 0, 'Real': 1}

In [37]:
result = model.predict(test_image)
predicted_class = np.argmax(result[0])


if predicted_class == 0:
    prediction = 'Fake'
else:
    prediction = 'Real'


print(f"The predicted class is: {prediction}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
The predicted class is: Fake


In [34]:
model.save('DeepFake_img_classifier.keras')